In [6]:
require 'hdf5'
require 'math'

f = hdf5.open("data.hdf5", "r")

X_train = f:read("windows_train"):all()
Y_train = f:read("train_Y"):all()
X_valid = f:read("windows_valid"):all()
Y_valid = f:read("valid_reduced_Y"):all()

nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]

f:close()

In [81]:
function createHash(longTensor) 
    local s = ""
    for i=1,longTensor:size(1) do
        s = s .. tostring(longTensor[i]) .. " "
    end
    return s
end

function add_padding(r, n, win) 
    if r < n then
        for i=1,n-r do
            win[i] = 30
        end
        r = r + 1
    end
    if win[n] == 30 then
        r = 1
    end
    return r, win
end

function count_train(windows_train, space_train, n_gram)
    local count_table = {}
    local space_table = {}
    local restart = 1
    for i=1,windows_train:size(1) do
        restart, padded = add_padding(restart, n_gram, windows_train[i])
        local key = createHash(padded)
        if count_table[key] then
            count_table[key] = count_table[key] + 1
        else
            count_table[key] = 2
            space_table[key] = 0
        end
        if space_train[i] == 2 then
            space_table[key] = space_table[key] + 1
        end
    end
    for k,_ in pairs(space_table) do
        space_table[k] = (space_table[k]+1)/(count_table[k])
    end
    return space_table
end  

function count_predict(lookup_table, windows_valid, space_valid, n_gram)
    local restart = 1
    local prob_vector = torch.Tensor(windows_valid:size(1))
    for i=1,windows_valid:size(1) do
        restart, padded = add_padding(restart, n_gram, windows_valid[i])
        local key = createHash(padded)
        if not lookup_table[key] then
            prob_vector[i] = 1/windows_valid:size(1)
        else
            prob_vector[i] = lookup_table[key]
        end
    end
    return prob_vector
end


In [77]:
win_size = X_train:size(2)
ngram_table = count_train(X_train, Y_train, win_size)

In [82]:
probabilities = count_predict(ngram_table, X_valid, Y_valid, win_size)

In [88]:
p = probabilities/probabilities:sum()

-14.9975
-14.2556
-12.9338
-11.7707
-14.1315
-13.8989
-13.7988
-10.5799
-10.2413
-14.2043
-12.9945
-12.3585
-10.2565
-14.7664
-11.2599
-12.3585
-12.2407
-10.7863
-13.7988
-13.9679
-10.1937
-15.7103
-14.9570
-13.1057
-12.1071
-10.8226
-13.7722
-13.7722
-14.5179
-10.3721
-15.6080
-10.1655
-15.6166
-12.6461
-12.1071
-11.5475
-14.1866
-13.2523
-13.2967
-13.2523
-13.2523
-10.2482
-10.1612
-13.7448
-12.9338
-13.8501
-12.4638
-13.3393
-12.2407
-13.7722
-13.8248
-10.1612
-14.3953
-11.2599
-11.2599
-13.4571
-14.9404
-11.4140
-10.3154
-11.9530
-12.1071
-10.2422
-14.9234
-15.9783
-15.9010
-10.2468
-18.7278
-15.7816
-10.1612
-11.5475
-11.5475
-21.8201
-21.8201
-11.2599
-15.5225
-15.5225
-15.5225
-15.5225
-10.2314
-18.7261
-16.4054
-16.4054
-16.4054
-18.7578
-10.2314
-18.7261
-15.7258
-13.9899
-11.9530
-13.5285
-11.0436
-11.9530
-12.4638
-10.1612
-14.8974
-11.1786
-13.3393
-21.8201
-21.8201
-21.8201
-21.8201
-13.1057
-13.9679
-13.0516
-13.4934
-13.3393
-13.3393
-10.6312
-12.9945
-12.2407
-11.7707
-

13.2523
-13.6876
-11.3111
-21.8201
-12.2947
-10.3421
-12.4638
-12.4638
-14.9529
-21.8201
-21.8201
-21.8201
-13.8248
-14.1125
-10.5290
-15.5272
-13.6876
-11.5475
-13.1057
-14.1866
-13.2523
-13.2967
-13.2523
-13.2523
-10.2482
-13.2523
-13.2967
-10.8544
-12.3585
-13.8748
-13.9899
-13.4934
-13.4934
-13.4934
-13.4934
-10.4749
-13.1057
-10.8544
-21.8201
-10.1612
-16.2157
-13.5952
-12.6461
-13.2967
-10.8946
-10.3154
-13.4934
-10.1612
-13.8248
-12.7262
-14.5680
-10.2238
-16.1274
-12.3585
-11.5475
-21.8201
-10.3436
-11.7707
-11.2599
-10.1612
-13.7166
-14.6721
-14.8974
-16.1401
-18.7578
-10.2314
-18.7261
-11.8388
-12.3322
-10.1612
-17.7363
-15.8650
-12.8003
-11.5475
-14.1866
-12.7262
-12.7262
-12.7262
-12.7262
-14.3201
-10.4081
-15.3260
-13.9454
-13.9679
-13.8989
-15.3977
-15.4545
-15.4187
-10.5135
-12.2407
-21.8201
-21.8201
-21.8201
-14.2043
-12.4567
-13.9899
-10.1656
-16.2968
-13.1057
-11.5475
-11.2599
-10.8544
-10.8544
-10.1612
-14.2043
-16.4528
-10.1986
-15.5083
-14.4787
-13.6577
-13.0516
-1

1.8201
-11.5475
-14.1125
-15.8137
-15.5319
-15.5639
-15.5818
-15.5684
-10.3126
-15.5862
-15.1029
-10.2739
-16.0584
-15.1719
-13.4934
-11.5475
-11.7887
-12.8003
-12.7639
-11.2834
-13.4193
-15.7707
-14.9529
-21.8201
-21.8201
-21.8201
-13.8248
-14.1125
-10.5290
-15.5272
-12.8003
-11.7707
-14.1686
-15.8482
-10.1646
-15.8482
-12.8003
-11.7707
-11.2599
-14.2884
-13.2967
-12.5591
-13.1057
-10.1612
-12.7262
-13.4571
-12.3585
-11.2599
-13.8748
-13.9899
-13.4934
-13.4934
-13.4934
-13.4934
-10.4749
-13.1057
-10.8544
-10.8544
-10.3883
-17.0947
-12.8003
-10.8544
-11.9530
-10.2078
-14.1866
-11.0775
-11.2599
-21.8201
-21.8201
-21.8201
-21.8201
-21.8201
-12.1071
-13.1570
-10.2302
-14.4239
-12.0330
-13.8869
-10.1634
-17.7363
-14.0531
-21.8201
-21.8201
-11.2599
-21.8201
-21.8201
-13.1057
-14.6156
-13.6876
-15.0288
-15.4842
-14.1315
-13.8989
-13.7988
-10.5799
-13.3801
-11.5475
-12.4638
-11.7707
-12.8693
-12.8003
-11.4140
-10.6721
-11.7707
-15.4444
-13.7448
-13.2523
-10.4797
-16.4170
-14.9570
-14.9653
-10

.2873
-16.4170
-10.1612
-13.4934
-10.8544
-13.7722
-13.7722
-18.7578
-10.2314
-18.7261
-15.8516
-13.6332
-13.8501
-16.7446
-14.7151
-14.7151
-14.6830
-10.6577
-10.3154
-14.9896
-13.5790
-10.5234
-16.2157
-10.8413
-12.3585
-12.8693
-14.8056
-14.9148
-14.9487
-11.6906
-10.3356
-14.6610
-13.5952
-14.2043
-14.1125
-13.5285
-13.5285
-13.5285
-11.1728
-14.6610
-11.7707
-10.1612
-13.2523
-14.0324
-21.8201
-21.8201
-21.8201
-21.8201
-21.8201
-21.8201
-21.8201
-21.8201
-21.8201
-21.8201
-21.8201
-21.8201
-21.8201
-14.6721
-14.6386
-14.5557
-10.8668
-14.8341
-13.2967
-11.2599
-13.2058
-13.1057
-13.2523
-11.2599
-11.2599
-14.8526
-12.5591
-12.5591
-12.5591
-10.2993
-16.0988
-12.3585
-12.8003
-12.5591
-10.8056
-13.1057
-10.8544
-10.8544
-10.8544
-10.8544
-11.9530
-12.9646
-13.8501
-10.3319
-15.4342
-13.3801
-11.5475
-11.6276
-13.8748
-13.4934
-13.7448
-10.4635
-10.4977
-12.3585
-13.8989
-13.9224
-14.9234
-10.1687
-16.1527
-10.7283
-12.2407
-13.7448
-10.1612
-15.4794
-12.4638
-10.5667
-12.6461
-11.

7261
-14.6830
-13.5624
-12.1071
-10.9057
-10.3436
-14.3659
-12.9338
-12.6036
-10.1634
-17.7363
-15.2111
-13.5952
-12.6461
-14.6721
-14.6386
-14.5557
-10.8668
-10.3263
-15.1448
-13.1570
-15.0957
-15.7410
-10.1652
-17.1282
-13.7166
-14.4239
-15.8380
-18.7578
-10.2314
-18.7261
-15.8516
-13.7988
-12.7262
-12.8003
-11.0085
-12.2407
-12.2407
-10.1612
-12.2407
-21.8201
-10.3010
-15.2238
-14.6499
-12.9338
-21.8201
-11.2599
-21.8201
-13.6270
-13.4934
-10.1612
-17.7363
-15.4891
-15.5549
-16.2412
-18.7578
-10.2314
-18.7261
-15.4187
-13.4934
-11.5475
-12.8003
-10.2666
-12.3585
-21.8201
-21.8201
-14.1125
-13.0516
-11.2599
-11.5475
-12.5591
-10.4797
-12.2407
-11.2599
-12.8693
-10.8544
-13.8248
-13.8248
-13.8248
-14.4379
-13.2058
-13.8501
-13.8501
-10.8568
-16.9077
-12.9338
-12.2407
-12.9945
-13.2673
-13.0516
-10.1612
-14.0114
-14.4920
-16.0362
-10.3741
-18.7278
-16.5191
-14.8708
-13.1570
-14.6721
-14.6386
-14.5557
-10.8668
-14.8341
-13.1057
-12.9338
-13.4193
-12.3585
-11.5475
-10.4042
-11.9530
-11.9

...

5
-13.2967
-10.8544
-10.1612
-11.5475
-12.7262
-12.7262
-13.0516
-10.2873
-16.4170
-13.4571
-12.5591
-12.4638
-13.7722
-13.8248
-13.8248
-13.8248
-10.6059
-13.4934
-11.6873
-12.8693
-10.1612
-17.7363
-14.6499
-13.9454
-13.9899
-15.4495
-10.2993
-16.0988
-12.5591
-13.3801
-11.9530
-10.5135
-13.9679
-10.8544
-21.8201
-11.0367
-13.8248
-13.9224
-13.9224
-10.9285
-10.2826
-13.8748
-11.5775
-13.1057
-12.8693
-12.7262
-14.2556
-13.1570
-14.2217
-14.5050
-10.1612
-16.4602
-16.4528
-10.1986
-15.5083
-14.5433
-11.6653
-10.2125
-16.1701
-12.8003
-12.6461
-12.7262
-11.1728
-13.4193
-10.1976
-14.3953
-11.9858
-12.8354
-12.1071
-13.6577
-13.7448
-10.4867
-13.4193
-12.6461
-14.2043
-12.3585
-15.1585
-15.1585
-11.2797
-14.7256
-10.8544
-21.8201
-13.1057
-15.7064
-10.1612
-15.7296
-10.9545
-13.5285
-11.7707
-12.7262
-12.2407
-14.3044
-14.7151
-14.7151
-10.5037
-14.6938
-13.6876
-10.2461
-15.5594
-16.8657
-16.7959
-10.2659
-18.7278
-16.6243
-16.1943
-16.1627
-16.7446
-16.2837
-16.1701
-10.2497
-15.8380


-14.1315
-14.9148
-14.8886
-16.0855
-11.2354
-14.9815
-13.9454
-10.3949
-14.6610
-13.3393
-13.5285
-12.3585
-13.2967
-13.8989
-13.2967
-10.2446
-13.2967
-12.6461
-10.1612
-16.4620
-10.1612
-14.7960
-10.2195
-16.7488
-16.0611
-16.3097
-16.2968
-15.6398
-16.3267
-16.6730
-10.3309
-14.6938
-12.2947
-10.3421
-15.6080
-14.0114
-16.5225
-10.2223
-18.7278
-16.2660
-15.5036
-15.3870
-14.2721
-13.9454
-13.9454
-13.9454
-10.7366
-14.3807
-11.2599
-11.2599
-14.1866
-10.9636
-15.7064
-12.3585
-10.3436
-12.4638
-13.9454
-13.9454
-10.1612
-15.5549
-10.8118
-12.8003
-12.9338
-14.0114
-11.4140
-10.5966
-13.6876
-13.2967
-10.1894
-16.4284
-10.1612
-14.7960
-10.2195
-16.7488
-16.0611
-16.3097
-16.2968
-15.6398
-16.3267
-16.6730
-10.3309
-16.5051
-15.9138
-10.2258
-14.4097
-10.1612
-14.3659
-10.1740
-16.7488
-10.5622
-14.7256
-14.1315
-14.9148
-14.8886
-16.0855
-11.2354
-14.9815
-13.9454
-10.3949
-14.6610
-13.3393
-13.5285
-12.3585
-13.2967
-13.8989
-13.2967
-10.2446
-13.2967
-12.6461
-10.1612
-16.4620


-15.7707
-14.9529
-14.9148
-14.8797
-10.5579
-14.6499
-13.8248
-13.6876
-13.8989
-10.9809
-10.2966
-15.1652
-13.2058
-10.3710
-15.0739
-13.2523
-11.2599
-11.9530
-14.0324
-14.1125
-14.3044
-10.8386
-14.3509
-12.9338
-11.5222
-13.4934
-15.7410
-15.7372
-13.9454
-14.8056
-14.8056
-14.8434
-10.4518
-10.2826
-15.8137
-10.8364
-10.4489
-14.5050
-12.1071
-13.7988
-13.8989
-15.1585
-15.1585
-11.2797
-14.5557
-10.2353
-14.3659
-14.0931
-10.1612
-15.5594
-15.0440
-14.9487
-10.3120
-18.7278
-16.0855
-14.2721
-11.6873
-14.6830
-14.7960
-14.7960
-10.1713
-15.4645
-13.0516
-14.2043
-13.6270
-14.8797
-13.6577
-13.7448
-11.0367
-13.1570
-11.9530
-11.7707
-13.4193
-14.6271
-10.2691
-14.5307
-13.1057
-10.1612
-14.6610
-10.6577
-14.4517
-13.8501
-15.8311
-15.9389
-18.7578
-10.2314
-18.7261
-15.4135
-13.3393
-12.1071
-13.4571
-10.1612
-12.5591
-21.8201
-12.8003
-11.0775
-11.8606
-10.1612
-17.7363
-14.6271
-11.6082
-11.7707
-12.8003
-12.9945
-12.9945
-12.9945
-12.9945
-10.5461
-14.1866
-12.7262
-11.7707
-

11.2354
-14.9815
-11.7539
-13.5624
-12.1071
-13.9454
-14.0324
-10.9636
-15.7064
-13.9224
-13.5952
-13.8989
-14.8974
-14.4920
-13.4193
-10.4534
-13.1570
-15.8815
-13.6270
-11.2599
-10.8544
-11.2354
-14.9815
-13.3801
-11.5475
-13.4571
-13.2673
-14.2884
-14.4097
-11.0390
-10.2207
-15.3087
-13.7166
-12.4638
-10.1656
-16.2968
-13.1057
-13.5952
-13.8989
-15.1585
-15.1585
-11.2797
-14.5557
-10.2353
-14.3659
-13.2967
-10.3816
-14.6610
-10.6577
-14.4517
-12.7262
-13.0516
-12.8003
-10.6031
-10.1612
-13.3801
-12.8693
-13.2523
-11.2599
-11.2599
-11.2599
-11.2599
-10.5667
-13.5952
-14.0931
-12.6461
-12.2407
-13.4571
-10.1612
-12.5591
-10.8544
-21.8201
-14.2217
-13.5285
-13.2058
-14.2556
-16.7446
-16.2837
-16.1701
-10.2497
-15.8380
-13.4193
-10.8544
-13.1570
-16.0611
-15.5365
-13.2523
-13.2523
-14.8526
-10.9144
-14.7462
-14.5050
-13.7166
-11.9530
-21.8201
-14.3807
-10.8842
-10.5290
-13.1570
-11.9530
-14.1125
-10.2734
-17.0947
-15.8346
-15.8516
-10.2532
-18.7278
-10.1612
-14.5307
-11.2599
-11.2599
-2

1.4830
-10.2994
-11.9530
-15.8815
-13.7448
-13.2523
-13.2523
-10.2421
-14.7256
-12.8693
-14.4920
-10.2734
-17.0947
-15.8346
-15.8516
-10.2532
-18.7278
-13.1570
-13.1570
-12.8003
-14.3044
-14.7151
-14.7151
-10.5037
-14.6938
-13.0516
-12.2407
-12.2407
-10.1741
-14.5050
-10.1612
-13.9679
-10.2327
-16.8282
-15.1917
-10.2125
-15.5036
-10.1612
-14.6271
-10.2327
-16.7488
-15.1719
-15.3762
-15.3707
-11.8735
-15.4394
-16.6730
-10.3309
-16.5051
-14.8797
-14.1502
-14.6386
-10.1631
-16.4303
-10.1612
-13.7988
-10.2139
-16.7488
-15.1719
-15.3762
-15.3707
-11.8735
-15.4394
-16.6730
-10.3309
-16.5051
-14.6721
-13.6270
-13.7166
-10.2873
-16.4170
-10.4891
-11.2599
-10.1612
-15.0440
-14.1315
-14.3659
-11.7707
-14.6721
-10.3260
-14.3659
-12.8693
-11.5475
-13.1570
-14.4920
-10.3023
-14.3509
-11.5475
-13.5285
-14.1502
-15.1241
-15.1719
-15.3707
-11.6986
-13.4934
-21.8201
-21.8201
-13.1057
-14.8434
-10.3795
-10.2482
-12.5591
-21.8201
-21.8201
-13.8989
-15.1585
-15.1585
-11.2797
-13.1570
-12.4638
-10.1612
-16